In [2]:
from os import path
import matplotlib.pyplot as plt
import re
from os.path import join
import pandas as pd
import numpy as np
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn import svm
from sklearn.ensemble import BaggingClassifier
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import StratifiedKFold
from collections import Counter
import pdb
from scipy.sparse import coo_matrix

mypath="/Users/chenlaishi/Documents/GitHub/Stat_628_2/data"
file="random100000w.csv"
data=pd.read_csv(join(mypath,file))
data=data.dropna()
stars=data['stars'].values.tolist()
texts=data['text'].values.tolist()

def balance_classes(x, y):
    freqs = Counter(y)
    max_allowable = freqs.most_common()[-1][1]
    num_added = {clss: 0 for clss in freqs.keys()}
    new_y = []
    new_x = []
    for i, yy in enumerate(y):
        if num_added[yy] < max_allowable:
            new_y.append(yy)
            new_x.append(x[i])
            num_added[yy] += 1
    return new_x, new_y

balanced_x, balanced_y = balance_classes(texts, stars)
txt_pos = [balanced_x[i] for i,j in enumerate(balanced_y) if j>3]
txt_neg = [balanced_x[i] for i,j in enumerate(balanced_y) if j<3]

In [3]:
def tokenize(sentence, grams):
    words = sentence.split()
    tokens = []
    for gram in grams:
        for i in range(len(words) - gram + 1):
            tokens += ["_*_".join(words[i:i+gram])]
    return tokens

def build_dict(txtlist, grams):
    dic = Counter()
    for sentence in txtlist:
        dic.update(tokenize(sentence, grams))
    return dic

def process_files(txtlist, dic, r, grams):
    j=0
    value=[]
    row=[]
    col=[]
    for l in txtlist:
        tokens = tokenize(l, grams)
        indexes = []

        for t in tokens:
            try:
                indexes.append(dic[t])
            except KeyError:
                pass
        indexes = list(set(indexes))
        indexes.sort()
        for i in indexes:
            value.append(r[i])
            row.append(j)
            col.append(i)
        j+=1
    row=np.asarray(row)
    col=np.asarray(col)
    val=np.asarray(value)
    output=coo_matrix((val,(row,col)),shape=(len(txtlist),len(dic)))
    return output
    

def compute_ratio(poscounts, negcounts, alpha=1):
    alltokens = list(set(list(poscounts.keys()) + list(negcounts.keys())))
    dic = dict((t, i) for i, t in enumerate(alltokens))
    d = len(dic)
    p, q = np.ones(d) * alpha , np.ones(d) * alpha
    for t in alltokens:
        p[dic[t]] += poscounts[t]
        q[dic[t]] += negcounts[t]
    p /= abs(p).sum()
    q /= abs(q).sum()
    r = np.log(p/q)
    return dic, r
 

ngram = [int(1)]
poscounts = build_dict(txt_pos, ngram)         
negcounts = build_dict(txt_neg, ngram)         
dic, r = compute_ratio(poscounts, negcounts)
out=process_files(balanced_x, dic, r, ngram)

    
    

In [3]:
##SVD
from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD(n_components=100, n_iter=7, random_state=42)
svd.fit(out)  
Xsvd=svd.transform(out)
print(Xsvd.shape)

(48835, 100)


In [ ]:
#train
y=np.asarray(balanced_y)
skf = StratifiedKFold(n_splits=5)
skf.get_n_splits(Xsvd, y)
mse=0
for train_index, test_index in skf.split(Xsvd, y):
    X_train, X_test = Xsvd[train_index], Xsvd[test_index]
    y_train, y_test = y[train_index], y[test_index]
    clf = BaggingClassifier(svm.LinearSVC())
    clf.fit(X_train, y_train)
    preds=clf.predict(X_test)
    mse+=mean_squared_error(y_test, preds)   
mse=mse/5
print(mse)

In [4]:
out.shape

(48835, 56988)